In [1]:
import psycopg2 as pg2
import pandas as pd
import pprint 
import string
import unicodedata
import numpy as np
import pickle

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Import pickled cleaned data from previous project

In [42]:
with open('../data/boardgamedata.pickle', 'rb') as f:
    worddata, numdata = pickle.load(f)

In [43]:
worddata.head()

,name,description,category,mechanic
game.id,,,,
1,Die Macher,Die Macher is a game about seven sequential po...,"[Economic, Negotiation, Political]","[Area Control, Area Influence, Auction/Bidding..."
2,Dragonmaster,Dragonmaster is a trick-taking card game based...,"[Card Game, Fantasy]",[Trick-taking]
3,Samurai,"Part of the Knizia tile-laying trilogy, Samura...","[Abstract Strategy, Medieval]","[Area Control, Area Influence, Hand Management..."
4,Tal der Könige,When you see the triangular box and the luxuri...,[Ancient],"[Action Point Allowance System, Area Control, ..."
11,Bohnanza,Bohnanza is the first in the Bohnanza family o...,"[Card Game, Farming, Negotiation]","[Hand Management, Set Collection, Trading]"


In [8]:
numdata.set_index('game.id', drop=True, inplace=True)

In [11]:
numdata.describe()

,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,details.playingtime,details.yearpublished,stats.average,stats.averageweight,stats.bayesaverage,stats.numcomments,stats.owned,stats.stddev,stats.trading,stats.usersrated,stats.wanting,stats.wishing
count,90397.000000,90397.000000,90397.000000,90397.000000,90397.000000,90397.000000,90397.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000,90400.000000
mean,5.732004,51.773731,7.121143,1.982533,46.104506,51.773731,1814.056584,4.309627,0.866140,1.146279,36.044502,234.105608,0.945540,7.860996,123.134192,9.596858,34.333208
std,53.449543,331.657193,5.056647,0.860299,316.313227,331.657193,580.656445,2.927143,1.155766,2.300701,235.494572,1483.897861,0.989744,36.531960,1048.474551,50.702656,224.203389
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,5.000000,0.000000,2.000000,10.000000,5.000000,1986.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,30.000000,8.000000,2.000000,30.000000,30.000000,2005.000000,5.400000,0.000000,0.000000,2.000000,8.000000,0.986687,0.000000,3.000000,0.000000,1.000000
75%,6.000000,60.000000,12.000000,2.000000,60.000000,60.000000,2012.000000,6.666670,1.750000,0.000000,9.000000,58.000000,1.535955,2.000000,18.000000,3.000000,8.000000
max,11299.000000,60120.000000,120.000000,50.000000,60120.000000,60120.000000,2019.000000,10.000000,5.000000,8.489660,13841.000000,95401.000000,4.500000,1858.000000,67655.000000,1838.000000,9082.000000


## Build LDA Model

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
lda = LatentDirichletAllocation(learning_method='online', n_jobs=-2, random_state=9)

In [34]:
#define common words in data to remove
#built from pervious project
sw = set(stopwords.words('english'))

COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"
               ]
sw.update(COMMON_WORDS)

In [17]:
#Try CountVectorizer for now

from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(stop_words=sw, max_df=0.85, min_df=2, max_features=1000)
word_vec = tf_vectorizer.fit_transform(worddata['description'])

In [18]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(learning_method='online', n_jobs=-2, random_state=1659)

In [19]:
lda.fit(word_vec)

LatentDirichletAllocation(learning_method='online', n_jobs=-2,
                          random_state=1659)

In [20]:
import joblib

joblib.dump(lda, '../data/lda_model.joblib')
joblib.dump(tf_vectorizer, '../data/tf_vec.joblib')

['../data/tf_vec.joblib']

In [21]:
def top_topic_features(model, feature_names, num_features=10):
    sorted_topics = feature_names[model.components_.argsort(axis=1)[:, ::-1][:, :num_features]]
    return sorted_topics

In [22]:
feature_names = np.array(tf_vectorizer.get_feature_names())
top_topic_features(lda, feature_names=feature_names)

array([['war', 'battle', 'rules', 'units', 'map', 'combat', 'army',
        'attack', 'counters', 'one'],
       ['tokens', 'fun', 'players', 'play', 'get', 'around', 'fast',
        'family', 'token', 'find'],
       ['players', 'rsquo', 'one', 'city', 'time', 'action', 'take',
        'may', 'end', 'must'],
       ['move', 'pieces', 'space', 'one', 'players', 'first', 'spaces',
        'two', 'piece', 'race'],
       ['dice', 'roll', 'die', '140', '231', 'bull', '184', 'rolls',
        'rolling', 'monopoly'],
       ['cards', '229', 'play', 'players', 'draw', 'played', 'pack',
        'pile', 'one', '136'],
       ['one', 'players', 'two', 'tiles', '208', 'number', 'first',
        'play', 'three', 'score'],
       ['team', 'ship', 'word', 'questions', 'answer', 'words',
        'question', 'ships', 'teams', 'players'],
       ['227', '129', '130', '131', '128', '230', '188', 'de', '139',
        '174'],
       ['new', 'expansion', 'set', 'rules', 'play', 'character', 'games',
      

#### Remove numbers from vectorizer

Implement preprocess function

In [23]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text

In [35]:
count_vectorizer = CountVectorizer(stop_words=sw, preprocessor=preprocess_text, max_df=0.85,
                                  min_df=2, max_features=1000)

In [36]:
word_vec_2 = count_vectorizer.fit_transform(worddata['description'])

In [37]:
lda2 = LatentDirichletAllocation(learning_method='online', n_jobs=-2, 
                                random_state=1659)
lda2.fit(word_vec_2)

LatentDirichletAllocation(learning_method='online', n_jobs=-2,
                          random_state=1659)

In [38]:
feature_names2 = np.array(count_vectorizer.get_feature_names())
top_topic_features(lda2, feature_names=feature_names2)


array([['team', 'city', 'tokens', 'money', 'resources', 'build', 'phase',
        'power', 'building', 'token'],
       ['dice', 'move', 'one', 'roll', 'die', 'first', 'pieces', 'space',
        'two', 'spaces'],
       ['new', 'rules', 'expansion', 'cards', 'set', 'games', 'includes',
        'ship', 'also', 'edition'],
       ['questions', 'answer', 'eacute', 'party', 'question', 'la',
        'guess', 'correct', 'correctly', 'variant'],
       ['de', 'fun', 'treasure', 'monster', 'friends', 'find', 'dark',
        'gold', 'items', 'dungeon'],
       ['time', 'get', 'one', 'make', 'like', 'word', 'many', 'way',
        'also', 'best'],
       ['cards', 'one', 'round', 'two', 'tiles', 'wins', 'number',
        'first', 'played', 'three'],
       ['cards', 'character', 'characters', 'use', 'heroes', 'mdash',
        'new', 'unique', 'action', 'abilities'],
       ['rsquo', 'rdquo', 'red', 'ldquo', 'ndash', 'bull', 'black',
        'blue', 'green', 'white'],
       ['war', 'battle', 'at

In [41]:
#pickle the fitted transformed model

with open('../models/lda.pickle', 'wb') as f:
    pickle.dump(lda2, f)

## Building the Recommender

In [ ]:
# write function to find index of game given the game title

def find_game_idx(df, game_title);
    return df.name[]